### Import Packages

In [1]:
import os
import json
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

from dataset import EvalDataset

### Selecting Weight of load model

In [2]:
weight = 1383

### Output as image with rectangle by model predict

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def load_model():
    model_path = os.path.join('model', f'faster-rcnn-{weight}.pkl')
    return torch.load(model_path)

def draw_ret(ax, box):
    assert len(box) == 4
    x, y = box[0], box[1]
    width, height = (box[2]-box[0]), (box[3]-box[1])
    rect = patches.Rectangle((x, y), width, height, linewidth=1,
                                     edgecolor='r', facecolor='none')
    ax.add_patch(rect)

@torch.no_grad()
def eval_out_image():
    image_dir = os.path.join('..', 'data', 'test')
    print(f'image_dir: {image_dir}')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'device: {device}')

    dataset = EvalDataset(image_dir)
    data_loader = DataLoader(dataset, batch_size=1)

    model = load_model()
    model.eval()
    model.to(device)

    until = 30

    pbar = tqdm(data_loader)
    for i, (image, image_name) in enumerate(pbar):
        image = image.to(device)
        output = model(image)
        # print(output)
        output = output[0]

        boxes = output['boxes']
        labels = output['labels']
        scores = output['scores']

        fig, ax = plt.subplots()
        title_message = [image_name[0]]
        ax.imshow(transforms.ToPILImage()(np.squeeze(image)))
        for box, label, score in zip(boxes, labels, scores):
            if score > 0.75:
                draw_ret(ax, box)
                title_message.append(f'{label}:{100*score:.0f}%')
        
        # print(title_message)
        ax.set_title(' '.join(title_message))

        plt.show()

        # if i+1 > until:
        #     break

In [ ]:
# eval_out_image()

### Draw rectangle by existed json

In [ ]:
def draw_rect_in_json():
    def draw_ret(ax, box):
        assert len(box) == 4
        x, y = box[0], box[1]
        width, height = box[2], box[3]
        rect = patches.Rectangle((x, y), width, height, linewidth=1,
                                        edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    from  PIL import Image
    json_path = os.path.join(f'answer-{weight}.json')
    with open(json_path, 'r') as json_file:
        answer = json.load(json_file)

    image_dir = os.path.join('..', 'data', 'test')
    print(f'image_dir: {image_dir}')

    for i, item in enumerate(answer):
        box = item['bbox']
        label = item['category_id']
        score = item['score']
        if score > 0.9:

            fig, ax = plt.subplots()
            title_message = [str(item['image_id'])]
            image = Image.open(os.path.join(image_dir, str(item['image_id'])+'.png'))
            ax.imshow(image)
            draw_ret(ax, box)
            print(box)
            title_message.append(f'{label}:{100*score:.0f}%')
            
            ax.set_title(' '.join(title_message))

            plt.show()

In [ ]:
# draw_rect_in_json()

In [ ]:
# assert False == True

### Output as json, by model predict

In [ ]:
@torch.no_grad()
def eval_out_json():
    answer = []

    image_dir = os.path.join('..', 'data', 'test')
    print(f'image_dir: {image_dir}')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'device: {device}')

    dataset = EvalDataset(image_dir)
    data_loader = DataLoader(dataset, batch_size=1)

    model = load_model()
    model.eval()
    model.to(device)

    # count = 0

    pbar = tqdm(data_loader)
    for i, (image, image_name) in enumerate(pbar):
        image = image.to(device)
        output = model(image)
        output = output[0]

        boxes = output['boxes']
        labels = output['labels']
        scores = output['scores']

        for box, label, score in zip(boxes, labels, scores):
            if score > 0.75:
                dict_ = {}
                dict_['image_id'] = image_name[0].split('.')[0]
                dict_['score'] = float(score)
                if int(label) < 10:
                    dict_['category_id'] = int(label)
                else:
                    dict_['category_id'] = 0
                # print(box)
                dict_['bbox'] = box
                # {
                #     "image_id": 117,
                #     "score": 0.9752130508422852,
                #     "category_id": 3,
                #     "bbox": [
                #         41.071231842041016,
                #         8.766018867492676,
                #         13.521903991699219,
                #         25.68875789642334
                #     ]
                # },
                answer.append(dict_)
        # count += 1
        # if count == 3:
        #     break
    return answer


In [ ]:
# answer = eval_out_json()

In [ ]:
answer[:4]

### Transform json format from (x1, y1, x2, y2) to (x1, y1, width, height)

In [ ]:
for r in tqdm(range(len(answer))):
    answer[r]['image_id'] = int(answer[r]['image_id'])
    answer[r]['bbox'] = list(answer[r]['bbox'])
    for i in range(4):
        answer[r]['bbox'][i] = float(answer[r]['bbox'][i].cpu())
    x1 = min(answer[r]['bbox'][0], answer[r]['bbox'][2])
    x2 = max(answer[r]['bbox'][0], answer[r]['bbox'][2])
    y1 = min(answer[r]['bbox'][1], answer[r]['bbox'][3])
    y2 = max(answer[r]['bbox'][1], answer[r]['bbox'][3])
    # width, height = (answer[r]['bbox'][2] - answer[r]['bbox'][0]), (answer[r]['bbox'][3] - answer[r]['bbox'][1])
    width, height = (x2 - x1), (y2 - y1)
    answer[r]['bbox'] = [x1, y1, width, height]

In [ ]:
answer = sorted(answer, key=lambda x: x['image_id'])

In [ ]:
answer[:4]

In [ ]:
import json
print(json.dumps(answer, indent=4), file=open(f'answer-{weight}.json', 'w'))

In [3]:
def filter_json():
    json_path = os.path.join(f'answer-{weight}.json')
    with open(json_path, 'r') as json_file:
        answer = json.load(json_file)
    
    filtered_answer = []

    score_filter = 0.9
    for item in tqdm(answer):
        if item['score'] > score_filter:
            filtered_answer.append(item)
    
    print(json.dumps(filtered_answer, indent=4), file=open(f'answer-{weight}_{score_filter}.json', 'w'))

In [4]:
filter_json()

100%|██████████| 28477/28477 [00:00<00:00, 3556279.25it/s]
